In [0]:
import requests
from datetime import datetime, timedelta

# Set your NASA API key
API_KEY = "XKTOxNMJihb4WoNKGd61IGN6vycTrq47lqveDe26"
BASE_URL = "https://api.nasa.gov/planetary/apod"

# Define date range
start_date = datetime(2025, 6, 29)
end_date = datetime(2025, 7, 3)

# Prepare list of dates
date_list = [(start_date + timedelta(days=i)).strftime('%Y-%m-%d')
             for i in range((end_date - start_date).days + 1)]

# List to hold API data
data_list = []

# Fetch data for each date
for date in date_list:
    params = {
        "api_key": API_KEY,
        "date": date
    }
    response = requests.get(BASE_URL, params=params)
    if response.status_code == 200:
        data = response.json()
        data_list.append({
            "date": data.get("date"),
            "title": data.get("title"),
            "explanation": data.get("explanation"),
            "url": data.get("url")
        })
        print(f"✅ Added data for {date}")
    else:
        print(f"⚠️ Failed to fetch data for {date}: {response.status_code}")

# Convert list of dicts into Spark DataFrame
df = spark.createDataFrame(data_list)

# Show the DataFrame
display(df)

# Save as Delta table in the default workspace
table_name = "nasa_apod_data"
df.write.format("delta").mode("overwrite").saveAsTable(table_name)

print(f"🎉 Data saved as Delta table: {table_name}")
